# Face Recognition Model - Serving Function
The function uses PyTorch classifier on top of an opencv deep learning model to encode and recognize faces in given image

In [1]:
#nuclio: ignore
import nuclio

### Install dependencies and set config

In [ ]:
%%nuclio cmd -c
pip install opencv-contrib-python
pip install imutils
pip install torch torchvision
pip install pandas
pip install v3io_frames
pip install scikit-build
pip install cmake==3.13.3
pip install face_recognition

In [ ]:
%nuclio config spec.build.baseImage = "python:3.6-jessie"

### Perform necessary imports

In [2]:
import importlib.util
import cv2
import face_recognition
import imutils
import json
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import random
import string
import v3io_frames as v3f
import os
import datetime
from pickle import load
import shutil

### Set function environment variables

In [5]:
%%nuclio env
MODELS_PATH = /User/demos/demos/faces/models.py
MODEL_PATH = /User/demos/demos/faces/artifacts/model.bst
DATA_PATH = /User/demos/demos/faces/dataset/
CLASSES_MAP = /User/demos/demos/faces/artifacts/idx2name.csv

%nuclio: setting 'MODELS_PATH' environment variable
%nuclio: setting 'MODEL_PATH' environment variable
%nuclio: setting 'DATA_PATH' environment variable
%nuclio: setting 'CLASSES_MAP' environment variable


In [6]:
%nuclio env V3IO_ACCESS_KEY=${V3IO_ACCESS_KEY}

%nuclio: setting 'V3IO_ACCESS_KEY' environment variable


### Model serving class

In [11]:
class PytorchModel(object):
    def __init__(self):
        self.name = 'model.bst'
        self.model_filepath = os.environ['MODEL_PATH']
        self.model = None
        self.ready = None
        self.classes = os.environ['CLASSES_MAP']
        self.device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    
    def load(self):
        input_dim = 128
        hidden_dim = 64
        output_dim = self.n_classes

        spec = importlib.util.spec_from_file_location('models', os.environ['MODELS_PATH'])
        models = importlib.util.module_from_spec(spec)
        spec.loader.exec_module(models)

        model = models.FeedForwardNeuralNetModel(input_dim, hidden_dim, output_dim)
        model.to(self.device)
        model = model.double()
        model.__dict__['_modules'] = load(open(self.model_filepath, 'rb')) 
        self.model = model
        self.ready = True
    
    def predict(self, context, data, confidence=0.8):
        
        # acquires all metadata 
        time = data['time']
        cam_name = data['camera']
        img_url = data['file_path']
        
        # prepares image for use
        with open(img_url, 'rb') as f:
            content = f.read()
        img_bytes = np.frombuffer(content, dtype=np.uint8)
        image = cv2.imdecode(img_bytes, flags=1)
        
        # converts image format to RGB for comptability with face_recognition library and resize for faster processing
        rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        rgb = imutils.resize(image, width=750)
        ratio = image.shape[1] / float(rgb.shape[1])
        
        #gets mapping from label to name and known encodings
        idx2name_df = pd.read_csv(self.classes).set_index('value')
        self.n_classes = len(idx2name_df)
        
        if not self.model:
            self.load()
        
        #locates faces in image and extracts embbeding vector for each face
        context.logger.info('recognizing faces')
        boxes = face_recognition.face_locations(rgb, model='hog')
        encodings = face_recognition.face_encodings(rgb, boxes)
        
        #determines if face is a clear match/ambiguous.
        names = []
        labels = []
        for encoding in encodings:
            name = 'unknown'
            label = 'unknown'
            enc_tensor = torch.tensor(encoding, device=self.device)
            self.model.to(self.device)
            out = self.model(enc_tensor)
            pred_n, pred_i = out.topk(1)
            distrib = F.softmax(out, dim=0)
            
            max_p, max_i = distrib.topk(1)
            if max_p.item() > confidence:
                label = pred_i.cpu()
                name = idx2name_df.loc[label]['name'].values[0].replace('_', ' ')
            names.append(name)
            labels.append(label)
            print(name)
        #frames client to save necessary data
        client = v3f.Client("framesd:8081", container="users")

        #draw boxes with name on the image and performs logic according to match/ambiguous 
        ret_list = []
        for ((top, right, bottom, left), name, encoding, label) in zip(boxes, names, encodings, labels):  

            #rescale the face coordinates
            top = int(top * ratio)
            right = int(right * ratio)
            bottom = int(bottom * ratio)
            left = int(left * ratio)

            #random string for unique name in our saved data
            rnd_tag = ''.join(random.choices(string.ascii_uppercase + string.digits, k=5))

            new_row = {}
            #saves all extracted data to kv.  
            new_row = {'c' + str(i).zfill(3): encoding[i] for i in range(128)}
            if (name != 'unknown'): 
                new_row['label'] = label
                new_row['fileName'] = name.replace(' ', '_')
            else:
                new_row['label'] = -1
                new_path = os.environ['DATA_PATH'] + 'label_pending/unknown_' + img_url.split('/')[-1] +'.jpg'
                shutil.move(img_url, new_path)
                img_url = new_path
                new_row['fileName'] = 'unknown_' + rnd_tag
            
            new_row['imgUrl'] = img_url
            new_row['camera'] = cam_name
            new_row['time'] = datetime.datetime.utcnow()
            new_row_df = pd.DataFrame(new_row, index=[0])
            new_row_df = new_row_df.set_index('fileName')
            print(new_row['fileName'])
            client.write(backend='kv', table='iguazio/demos/demos/faces/artifacts/encodings', dfs=new_row_df) #, save_mode='createNewItemsOnly')

            #appends box and name to the returned list            
            ret_list.append(((top, right, bottom, left), name, max_p.item()))

        return ret_list

### Main function
simply initializes the model class and invokes the predict method 

In [12]:
model = PytorchModel()

def handler(context, event):
    
    return model.predict(context=context, data=event.body)

In [ ]:
#nuclio: end-code

### Set configuration for deployment

In [13]:
# converts the notebook code to deployable function with configurations
from mlrun import code_to_function, mount_v3io
fn = code_to_function('recognize-faces', runtime='nuclio')

# set the API/trigger, attach the home dir to the function
fn.with_http(workers=2).apply(mount_v3io())

# set environment variables
fn.set_env('MODELS_PATH', '/User/demos/demos/faces/models.py')
fn.set_env('MODEL_PATH', '/User/demos/demos/faces/artifacts/model.bst')
fn.set_env('CLASSES_MAP', '/User/demos/demos/faces/artifacts/idx2name.csv')
fn.set_env('V3IO_ACCESS_KEY', os.environ['V3IO_ACCESS_KEY'])

### Deploy the function to the cluster
May take a few minutes due to building of image

In [14]:
addr = fn.deploy(project='default')

[mlrun] 2020-02-23 14:37:14,011 deploy started
[nuclio] 2020-02-23 14:37:15,079 (info) Building processor image
[nuclio] 2020-02-23 14:37:20,126 (info) Build complete
[nuclio] 2020-02-23 14:37:26,182 (info) Function deploy complete
[nuclio] 2020-02-23 14:37:26,188 done updating recognize-faces, function address: 18.190.150.111:32760


In [ ]:
import nuclio_sdk
import logging

os.environ['MODEL_PATH']='/User/demos/demos/faces/artifacts/model.bst'
os.environ['CLASSES_MAP']='/User/demos/demos/faces/artifacts/idx2name.csv'
os.environ['MODELS_PATH']='/User/demos/demos/faces/models.py'
os.environ['DATA_PATH']='/User/demos/demos/faces/dataset/'

logger = logging.Logger('triggering')
ctx = nuclio_sdk.Context(logger=logger)

payload = {'file_path': 'pp.jpeg', 'time': '20191110131130', 'camera': 'cammy'}
ev = nuclio_sdk.Event(body=payload)
handler(ctx,ev)